In [1]:
import numpy as np

In [99]:
# Ground stae of 3 Qubit system 

psi1,psi2 = [0]*8
psi1[0] = 1
psi2[2] = 1

In [100]:
psi1

[1, 0, 1, 0, 0, 0, 0, 0]

In [101]:
psi2

[1, 0, 1, 0, 0, 0, 0, 0]

In [84]:
X = np.array([[0,1],[1,0]])
I = np.identity(2)

O1 = np.kron(np.kron(I,I),X)
O2 = np.kron(np.kron(I,I),X)
              
              

In [85]:
O1@psi

array([0., 1., 0., 0., 0., 0., 0., 0.])

In [86]:
O2@psi

array([0., 1., 0., 0., 0., 0., 0., 0.])

In [78]:
np.kron(X,np.kron(I,I))

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.]])